In [1]:
import time

In [81]:
def lzw_encode(input_file, max_dict_size):
    # Inicializa o dicionário LZW com todos os caracteres ASCII possíveis
    dict_size = 256
    dictionary = {bytes([i]): i for i in range(dict_size)}
    # next_code = 256
    current_sequence = bytes()
    
    with open(input_file, "rb") as file, open("output.bin", "wb") as output_file:
        # Codifica cada símbolo da entrada
        while True:
            symbol = file.read(1)
            if not symbol:
                break
                
            # Acrescenta o próximo símbolo à sequência atual
            next_sequence = current_sequence + symbol # BYTE
            # Se a próxima sequência estiver no dicionário, continua acrescentando símbolos
            # até chegar a uma sequência que não esteja no dicionário
            if next_sequence in dictionary:
                current_sequence = next_sequence
            else:
                # Codifica a sequência atual e adiciona a nova sequência ao dicionário
                encoded_sequence = dictionary[current_sequence]
                #full_encode.append(bin(encoded_sequence)[2:].zfill(16))
                output_file.write(encoded_sequence.to_bytes(2, byteorder="big"))
                
                if dict_size < 2 ** max_dict_size:
                    dictionary[next_sequence] = dict_size
                    dict_size += 1
                # Começa uma nova sequência com o último símbolo lido
                current_sequence = symbol

        # Escreve o último código LZW no arquivo de saída
        if current_sequence:
            encoded_sequence = dictionary[current_sequence]
            output_file.write(encoded_sequence.to_bytes(2, byteorder="big"))
            
    return "disco_output.bin"



In [82]:
lzw_encode(r'J:\UFPB\TEORIA DA INFO\Trabalho\encode-decode-lzw\files\corpus16MB.txt', 16)

'disco_output.bin'

In [83]:
def lzw_decode(input_file, max_dict_size):
    # Inicializa o dicionário LZW com todos os caracteres ASCII possíveis
    dict_size = 256
    dictionary = {i: bytes([i]) for i in range(dict_size)}
    
    with open(input_file, "rb") as file, open("output.txt", "wb") as output_file:
        # Decodifica cada código LZW
        current_code = int.from_bytes(file.read(2), byteorder="big")
        current_sequence = dictionary[current_code]
        output_file.write(current_sequence)

        while True:
            next_code_bytes = file.read(2)
            if not next_code_bytes:
                break
                
            next_code = int.from_bytes(next_code_bytes, byteorder="big")
            # Se o próximo código LZW estiver no dicionário, decodifica-o
            if next_code in dictionary:
                next_sequence = dictionary[next_code]
            # Senão, decodifica a sequência atual acrescida do primeiro caractere da próxima sequência
            else:
                next_sequence = current_sequence + bytes([current_sequence[0]])

            # Escreve a sequência decodificada no arquivo de saída
            output_file.write(next_sequence)

            # Adiciona a sequência atual acrescida do primeiro caractere da próxima sequência ao dicionário
            if dict_size < 2 ** max_dict_size:
                dictionary[dict_size] = current_sequence + bytes([next_sequence[0]])
                dict_size += 1

            # Atualiza a sequência atual
            current_sequence = next_sequence

    return "disco_output.txt"


In [84]:
lzw_decode(r'J:\UFPB\TEORIA DA INFO\Trabalho\encode-decode-lzw\output.bin', 16)

'disco_output.txt'